In [1]:
from langchain_docling import DoclingLoader

c:\Anaconda3\envs\PaperGraph\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sourece = "ReAct.pdf"
loader = DoclingLoader(
    file_path=sourece,
    export_type = "markdown"
)
documents = loader.load()

The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
[INFO] 2026-01-21 19:29:57,835 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-21 19:29:57,849 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-21 19:29:57,863 [RapidOCR] download_file.py:60: File exists and is valid: C:\Anaconda3\envs\PaperGraph\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-21 19:29:57,864 [RapidOCR] main.py:50: Using C:\Anaconda3\envs\PaperGraph\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-21 19:29:58,408 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-21 19:29:58,409 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-21 19:29:58,412 [RapidOCR] download_file.py:60: File exists and is valid: C:\Anacond

In [3]:
print(documents[0].page_content[:500])  # Print the first 500 characters of the first document

## REACT : SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS

Shunyu Yao ∗ *,1 , Jeffrey Zhao 2 , Dian Yu 2 , Nan Du 2 , Izhak Shafran 2 , Karthik Narasimhan 1 , Yuan Cao 2

1 Department of Computer Science, Princeton University 2 Google Research, Brain team

1 {shunyuy,karthikn}@princeton.edu 2 {jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com

## ABSTRACT

While large language models (LLMs) have demonstrated impressive performance across tasks in language understanding and interactive deci


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Docling 마크다운 출력용 섹션 단위 분할
text_splitter = RecursiveCharacterTextSplitter(
    # 청크 최대 크기: 1200자
    # - 영어 논문 1200자 ≈ 300토큰 (대부분 임베딩 모델 512토큰 제한 내)
    # - 논문 섹션(Abstract, Introduction) 전체를 한 청크로 유지 가능
    # - 너무 작으면 문맥 손실, 너무 크면 검색 정확도 하락
    chunk_size=1200,
    
    # 청크 간 겹치는 부분: 200자 (chunk_size의 16%)
    # - 섹션 경계에서 문맥이 끊기는 것 방지
    # - 검색 시 경계 부근 정보 놓치지 않도록
    # - 일반적으로 chunk_size의 10-20% 권장
    chunk_overlap=200,
    
    # 분할 우선순위 (위에서부터 순서대로 시도)
    separators=[
        "\n## ",      # 1순위: 논문 섹션 헤더 (## Abstract, ## Introduction)
                      # Docling이 주요 섹션을 2레벨 헤더로 변환하므로
                      # 가장 큰 의미 단위인 섹션을 먼저 보존
        
        "\n### ",     # 2순위: 서브섹션 (### 3.1 Dataset, ### 3.2 Model)
                      # 섹션이 1200자 초과 시 서브섹션 단위로 자연스럽게 분할
        
        "\n\n",       # 3순위: 문단 경계
                      # 서브섹션도 길면 문단 단위로 분할
                      # 논문은 문단이 논리적 단위이므로 여기서 자르는 게 자연스러움
        
        "\n",         # 4순위: 줄바꿈
                      # 문단도 길면 줄 단위로
                      # 수식, 리스트, 코드 블록 등은 줄바꿈으로 구분됨
        
        ". ",         # 5순위: 문장 끝 (마침표 + 공백)
                      # 최소한 완전한 문장은 유지
                      # 공백까지 포함해야 문장 끝 정확히 감지
        
        " ",          # 6순위: 단어 경계 (공백)
                      # 문장도 초과하면 단어 단위로
                      # 최소한 단어는 쪼개지지 않게
        
        ""            # 7순위: 강제 분할 (문자 단위)
                      # 긴 URL, 코드, 수식 등 chunk_size 초과 시 문자로 강제 분할
                      # 에러 방지용 최후의 안전장치
    ],
    
    # 원본 문서에서의 시작 위치를 메타데이터에 추가
    # - RAG 검색 후 출처 추적 가능 (몇 번째 문자/페이지에서 왔는지)
    # - chunk.metadata['start_index']로 접근
    # - 사용자에게 "논문 3페이지 Introduction 섹션" 같은 정보 제공 가능
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)

In [7]:
print(chunks[2])

page_content='While large language models (LLMs) have demonstrated impressive performance across tasks in language understanding and interactive decision making, their abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action plan generation) have primarily been studied as separate topics. In this paper, we explore the use of LLMs to generate both reasoning traces and task-specific actions in an interleaved manner, allowing for greater synergy between the two: reasoning traces help the model induce, track, and update action plans as well as handle exceptions, while actions allow it to interface with and gather additional information from external sources such as knowledge bases or environments. We apply our approach, named ReAct , to a diverse set of language and decision making tasks and demonstrate its effectiveness over state-of-the-art baselines in addition to improved human interpretability and trustworthiness' metadata={'source': 'ReAct.pdf', 'start_index'